In [161]:
import pylab
import shutil 
import skimage.morphology as morph
import skimage.feature.texture as texture
import skimage.feature as feature
import skimage.filters as filters
import skimage.transform as transform
import skimage.draw
import skimage.util as utils
from skimage.restoration import denoise_bilateral
from skimage.io import imread
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel,RFE,SelectKBest,chi2
from sklearn.decomposition import PCA,FastICA
from sklearn.compose import make_column_transformer
from scipy.stats import rv_histogram

In [182]:
root ='/home/marcelo/PV_Cell/' # Caminho para imagens

fnamesF = [] # Nome das imagens PV sem defeito
with open(root+"Sem_Defeito.txt") as f:
 fnamesF = [s.strip() for s in f]

fnamesT = [] # Nome das imagens PV com defeito
with open(root+"Defeito.txt") as f:
 fnamesT =  [s.strip() for s in f]

# Abre arquivos de imagens em coloca em listas separadas
imF = [imread(root+"Sem_Defeito/"+f) for f in fnamesF]
imT = [imread(root+"Defeito/"+f) for f in fnamesT]

im = {-1:imF,1:imT}

roi_1 = [(10,25),(43,265)]
roi_2 = [(60,15),(140,280)]
roi_3 = [(156,15),(238,280)]
roi_4 = [(255,25),(290,265)]

In [183]:

def glsm_features(img,w = [4,8,16,32,64],theta = pylab.linspace(0,2*pylab.pi,10)):
  g = texture.greycomatrix(utils.img_as_ubyte(img), w, theta, 
                           levels=256, normed=True, symmetric=False)
  l = []
  for f in ['contrast','dissimilarity','homogeneity','energy','correlation']:
    l.append(texture.greycoprops(g,f))
  return l

def LBP_features(img,radius,method):
 h = []   
 for r in radius:
  lbp = feature.local_binary_pattern(img, 8 * r, r, method)
  n_bins = int(lbp.max() + 1)
  h.append(pylab.histogram(lbp, density=True, bins=n_bins, range=(0, n_bins)))
 return h

 

In [184]:
X1 =[]
lbl = []
i = 0
for label,img_list in im.items():
    for img in img_list:  
     i = i+1  
     print(i,' ',end='')
     aux1 = []  
     for roi in [roi_1,roi_2,roi_3,roi_4]: 
      x0,y0 = roi[0]
      x1,y1 = roi[1]
      aux1.append(glsm_features(img[x0:x1,y0:y1]))
     lbl.append(label)
     X1.append(aux1)
           

1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200  201  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216  217  218  219  220  221  222

In [185]:
import pickle

#f = open("/home/marcelo/PV_Cell/features2.pkl","wb")
#pickle.dump(X1,f)
x1 = []
for x in X1:
 #print(pylab.array(x[0]).shape)   
 aux = pylab.array(x).ravel().tolist()
#aux2 =[PCA(n_components=3,whiten=False).fit_transform(i) for i in aux]
 x1.append(aux)

x1 = pylab.array(x1)
print(x1.shape)
y = lbl

(472, 1000)


In [186]:
clf1 = RandomForestClassifier(n_estimators=150,min_samples_split=5,n_jobs=-1)
clf2 = SVC(kernel='linear',probability=True)
clf3 = LogisticRegression(max_iter=550)
clf4 = GaussianNB()
clf5 = KNeighborsClassifier(n_neighbors = 3)
clf6 = VotingClassifier(estimators=[('rfc',clf1),('svm',clf2),('lr',clf3)],voting='soft')

In [202]:
from sklearn.metrics import cohen_kappa_score as kappa_score
from sklearn.metrics import make_scorer

N = x1.shape[1]
# ["roc_auc","accuracy","precision","recall","f1",make_scorer(kappa_score)]
for n in range(70,800,10):   
  print(n,end='')
  #ct = make_column_transformer((SelectKBest(k=3000),slice(0,N1)),remainder=SelectKBest(k=1200))
  for clf in [clf1,clf2,clf3,clf4,clf5,clf6]:
   p = Pipeline([('Scaler',StandardScaler()),("Sel",SelectKBest(k=n)),("Classification",clf)]) 
   score = cross_val_score(p,x1,lbl, cv=StratifiedKFold(n_splits=10,shuffle=True),scoring="f1",n_jobs=-1)
   print(" {0:.2f} {1:.2f}".format(score.mean(),score.std()),end='')
  print()
 

70 0.87 0.05 0.87 0.04 0.87 0.05 0.84 0.04 0.82 0.06 0.89 0.05
80 0.88 0.05 0.86 0.05 0.87 0.07 0.83 0.06 0.83 0.05 0.88 0.05
90 0.88 0.03 0.88 0.05 0.88 0.05 0.83 0.06 0.83 0.04 0.88 0.05
100

KeyboardInterrupt: 

In [ ]:
### img = imT[10]
imr = imread(root+"cell_msk.png",as_gray=True)

#imr_g = filters.gaussian(imr,sigma = 2.) 
#img_g = filters.gaussian(img,sigma = .5) 
img_g =denoise_bilateral(img,sigma_spatial = 5.) 
#imr_t = imr_g > filters.threshold_otsu(imr_g)
img_t = img_g > filters.threshold_otsu(img_g)

msk = pylab.logical_xor(imr,img_t)
#msk = morph.binary_opening(msk,morph.rectangle(20,5)) 
#msk = morph.binary_closing(msk,morph.disk(5))    
#

pylab.figure(i,figsize=(10,20))
pylab.subplot(121)
pylab.imshow(img,'gray')
pylab.subplot(122)
pylab.imshow(img_t, 'gray')
 